# Introduction

The purpose of this notebook is to present the basics of Spark: why we use it, and what kind of operations we can perform using it.

In this notebook we cover the following topics:
- SparkContext object
- RDDs
- Operations on RDDs:
    - transformations
    - actions
- DataFrames

## SparkContext object

First of all, we need to create a Spark application. We do so with the help of the `SparkContext` object, which 

![Cluster mode overview](./data/cluster-overview.png)
Image source: https://spark.apache.org/docs/latest/cluster-overview.html



In [2]:
import sys
from random import random
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("Spark Intro")
sc = SparkContext.getOrCreate(conf=conf) # Alternatively, use SparkContext(conf=conf)

## Resilient Distributed Datasets - RDDs